In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_647976/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_647976/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_647976/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_647976/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs
0,\GRTUSDT\plots\2022-02-23T153035\backtest_resu...,5.4,GRTUSDT,True,14.060,0.0654,100.0,0.9689,True,9.442,0.0456,100.0,0.9847
1,\GRTUSDT\plots\2022-02-25T222300\backtest_resu...,5.5,GRTUSDT,True,15.000,0.0696,100.0,0.9691,True,9.567,0.0462,100.0,0.9847
2,\GRTUSDT\plots\2022-02-25T232504\backtest_resu...,5.5,GRTUSDT,True,15.000,0.0696,100.0,0.9691,True,9.567,0.0462,100.0,0.9847
3,\GRTUSDT\plots\2022-02-26T230933\backtest_resu...,5.5,GRTUSDT,True,15.010,0.0639,100.0,0.9163,True,21.340,0.0984,100.0,0.9189
4,\GRTUSDT\plots\2022-02-26T231251\backtest_resu...,5.5,GRTUSDT,True,16.920,0.0661,100.0,0.8871,True,27.040,0.1240,100.0,0.8681
...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,\ZECUSDT\plots\2022-02-25T222007\backtest_resu...,5.5,ZECUSDT,True,12.520,0.0599,100.0,0.9660,True,3.435,0.0173,100.0,0.9792
385,\ZECUSDT\plots\2022-02-25T232208\backtest_resu...,5.5,ZECUSDT,True,12.520,0.0599,100.0,0.9660,True,3.435,0.0173,100.0,0.9792
386,\ZENUSDT\plots\2022-03-05T072906\backtest_resu...,5.5,ZENUSDT,True,8.481,0.0372,100.0,0.9209,True,19.940,0.0882,100.0,0.8434
387,\ZILUSDT\plots\2022-03-06T080955\backtest_resu...,5.5,ZILUSDT,True,12.050,0.0533,100.0,0.9544,True,15.840,0.0657,100.0,0.9504


Now filter the dataframe

In [8]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["TOMOUSDT","TRBUSDT","KNCUSDT","STMXUSDT"]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              & (big_data['adgt_%'] > 0.09)
              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)
              & (big_data['ebrs'] > .85)
              & (big_data['ebrl'] > .85)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs,adgt_%
53,\KNCUSDT\plots\2022-03-06T230003\backtest_resu...,5.5,KNCUSDT,True,21.49,0.0915,100.0,0.9401,True,19.23,0.0593,100.0,0.9337,0.1508
264,\STMXUSDT\plots\2022-03-06T230405\backtest_res...,5.5,STMXUSDT,True,18.28,0.0791,100.0,0.9591,True,24.56,0.1010,100.0,0.8623,0.1801
302,\TOMOUSDT\plots\2022-03-06T225102\backtest_res...,5.5,TOMOUSDT,True,11.28,0.0505,100.0,0.8997,True,21.20,0.0643,100.0,0.9411,0.1148
303,\TRBUSDT\plots\2022-03-06T225523\backtest_resu...,5.5,TRBUSDT,True,20.07,0.0850,100.0,0.9287,True,26.96,0.1080,100.0,0.9323,0.1930


In [9]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_647976/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
53,\KNCUSDT\plots\2022-03-06T230003\backtest_resu...,KNCUSDT,0.1508,0.0915,0.0593,100.0,100.0,0.9401,0.9337
264,\STMXUSDT\plots\2022-03-06T230405\backtest_res...,STMXUSDT,0.1801,0.0791,0.1010,100.0,100.0,0.9591,0.8623
302,\TOMOUSDT\plots\2022-03-06T225102\backtest_res...,TOMOUSDT,0.1148,0.0505,0.0643,100.0,100.0,0.8997,0.9411
303,\TRBUSDT\plots\2022-03-06T225523\backtest_resu...,TRBUSDT,0.1930,0.0850,0.1080,100.0,100.0,0.9287,0.9323
